In [2]:
from experta import *
import math


class States(Fact):
    nm_Biome = Field(str)
    nm_Climate = Field(str)
    sn_Coast = Field(str)
    sn_Border = Field(str)
    sn_Population = Field(str)
    sn_COVID = Field(str)
    sn_PIB = Field(str)

    def Filter_Array(ds_Filter, nr_Index, objArrayEstados):
        print(ds_Filter, nr_Index, objArrayEstados)
        return list(filter(lambda x: ds_Filter == x[nr_Index], objArrayEstados))

    def Get_Medium_Value(objArrayEstados, nr_Index):
        objArrayEstados.sort(key=lambda x: x[nr_Index])

        return objArrayEstados[math.ceil(len(objArrayEstados)/2) - 1][nr_Index] + 1

    def Get_States():
        return [
            ['Acre', 'Amazonia', 'Equatorial', 'N', 'S', 15331, 906876, 1816],
            ['Alagoas', 'Caatinga', 'Semiárido', 'S', 'N', 54413, 3365351, 6141],
            ['Amapá', 'Amazonia', 'Equatorial', 'S', 'S', 16795, 877613, 1962],
            ['Amazonas', 'Amazonia', 'Equatorial',
                'N', 'S', 100109, 4269995, 13705],
            ['Bahia', 'Mata Atlântica', 'Tropical',
                'S', 'N', 286240, 14985284, 26650],
            ['Ceará', 'Caatinga', 'Semiárido', 'S', 'N', 155904, 9240580, 24127],
            ['Distrito Federal', 'Cerrado', 'Tropical',
             'N', 'N', 254817, 3094325, 10222],
            ['Espiríto Santo', 'Mata Atlântica', 'Tropical',
             'S', 'N', 137020, 4108508, 12369],
            ['Goiás', 'Cerrado', 'Tropical', 'N', 'N', 195682, 7206589, 22974],
            ['Maranhão', 'Amazonia', 'Equatorial',
                'S', 'N', 98179, 7153262, 10106],
            ['Mato Grosso', 'Pantanal', 'Equatorial',
             'N', 'S', 137443, 3567234, 13655],
            ['Mato Grosso do Sul', 'Pantanal', 'Tropical',
             'N', 'S', 106969, 2839188, 9468],
            ['Minas Gerais', 'Cerrado', 'Tropical',
                'N', 'N', 614876, 21411923, 53698],
            ['Pará', 'Amazonia', 'Equatorial', 'S', 'S', 440029, 8777124, 38116],
            ['Paraíba', 'Caatinga', 'Semiárido', 'S', 'N', 64374, 4059905, 9246],
            ['Paraná', 'Mata Atlântica', 'Subtropical',
             'S', 'S', 161350, 11597484, 16544],
            ['Pernambuco', 'Caatinga', 'Semiárido',
                'S', 'N', 186352, 9674793, 19552],
            ['Piauí', 'Caatinga', 'Semiárido', 'S', 'N', 50378, 3289290, 6980],
            ['Rio de Janeiro' 'Mata Atlântica', 'Tropical',
             'S', 'N', 758859, 17463349, 63880],
            ['Rio Grande do Norte', 'Caatinga', 'Semiárido',
             'S', 'N', 66970, 3560903, 7302],
            ['Rio Grande do Sul', 'Pampa', 'Subtropical',
             'S', 'S', 457294, 11466630, 34462],
            ['Rondônia', 'Amazonia', 'Equatorial', 'N', 'S', 44914, 1815278, 6506],
            ['Roraima', 'Amazonia', 'Equatorial', 'N', 'S', 13370, 652713, 1968],
            ['Santa Catarina', 'Mata Atlântica', 'Subtropical',
             'S', 'S', 298227, 7338473, 18953],
            ['São Paulo', 'Mata Atlântica', 'Tropical',
             'S', 'N', 2210562, 46649132, 147258],
            ['Sergipe', 'Caatinga', 'Semiárido', 'S', 'N', 42018, 2338474, 6003],
            ['Tocantins', 'Cerrado', 'Tropical', 'N', 'N', 35666, 1607363, 3716],
        ]


class Classificator(KnowledgeEngine):

    objArrayEstados = States.Get_States()

    def Verificar_Estados(self):
        if (len(self.objArrayEstados) == 0):
            print('Nenhum estado foi encontrado')
            self.reset()
            self.declare(Fact(action='restart'))
            self.run()


    @Rule(Fact(action='restart'))
    def Get_Restart_Question(self):
        self.objArrayEstados = States.Get_States()
        
        if(input("Deseja identificar outro Estado (S/N)?") == 'S'):
            self.reset()
            self.declare(Fact(action='start'))
            self.run()
        else:
            self.declare(Fact(end=True))


    ################################################# Biomas ################################################

    @Rule(Fact(action='start'))
    def Get_Biome_Question(self):
        nm_Biome = input(
            'Qual é o bioma predominante no seu Estado? (Amazonia - Caatinga - Mata Pantanal - Cerrado - Pampa)')

        self.objArrayEstados = States.Filter_Array(
            nm_Biome, 1, self.objArrayEstados)

        self.Verificar_Estados()

        self.declare(States(nm_Biome=nm_Biome))

    @Rule(States(nm_Biome='Pampa'))
    def Print_Rio_Grande_do_Sul(self):
        print('O estado em que você está pensando é o Rio Grande do Sul')
        self.declare(Fact(action='restart'))

    ################################################# Climate ################################################

    @Rule(AND(States(nm_Biome=W()), NOT(States(nm_Climate=W()))))
    def Get_Climate_Question(self):
        nm_Climate = input(
            'Qual é o clima predominante no seu Estado? (Equatorial - Semiárido - Tropical - Subtropical)\n')

        self.objArrayEstados = States.Filter_Array(
            nm_Climate, 2, self.objArrayEstados)

        self.Verificar_Estados()

        self.declare(States(nm_Climate=nm_Climate))

    @Rule(AND(States(nm_Biome='Pantanal'),
          States(nm_Climate="Equatorial"),
              ))
    def Print_Mato_Grosso(self):
        print('O estado em que você está pensando é o Mato Grosso')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Pantanal'),
              States(nm_Climate="Tropical"),
              ))
    def Print_Mato_Grosso_do_Sul(self):
        print('O estado em que você está pensando é o Mato Grosso do Sul')
        self.declare(Fact(action='restart'))

    ################################################# Coast ################################################

    @Rule(AND(States(nm_Biome=W()),
              States(nm_Climate=W()),
              NOT(States(sn_Coast=W()))))
    def Get_Coast_Question(self):

        sn_Coast = input(
            'Por favor, responda com S ou N se o seu estado é litorâneo:\n')

        self.objArrayEstados = States.Filter_Array(
            sn_Coast, 3, self.objArrayEstados)

        self.Verificar_Estados()

        self.declare(States(sn_Coast=sn_Coast))

    ################################################# Border ################################################

    @Rule(AND(
        (States(nm_Biome=W())),
        (States(nm_Climate=W())),
        (States(sn_Coast=W())),
        NOT(States(nm_Border=W()))
    ))
    def Get_Border_Question(self):

        sn_Border = input(
            'Por favor, responda com S ou N se o seu estado é fronteira com algum país:\n')

        self.objArrayEstados = States.Filter_Array(
            sn_Border, 4, self.objArrayEstados)

        self.Verificar_Estados()

        self.declare(States(sn_Border=sn_Border))

    @Rule(AND(States(nm_Biome='Amazonia'),
          States(nm_Climate="Equatorial"),
          States(sn_Coast="S"),
          States(sn_Border="N"),
              ))
    def Print_Maranhao(self):
        print('O estado em que você está pensando é o Maranhão')
        self.declare(Fact(action='restart'))

    ################################################# PIB ################################################

    @Rule(AND(
        (States(nm_Biome=W())),
        (States(nm_Climate=W())),
        (States(sn_Coast=W())),
        (States(sn_Border=W())),
        NOT(States(sn_PIB=W())),
    ))
    def Get_PIB_Question(self):
        vl_PIB = States.Get_Medium_Value(self.objArrayEstados, 5)

        sn_PIB = input(
            'O seu estado tem o PIB maior que {0}?\n'.format(vl_PIB))

        self.objArrayEstados = list(filter(lambda x: x[5] > vl_PIB,  self.objArrayEstados) if sn_PIB == 'S' else filter(
            lambda x: x[5] < vl_PIB,  self.objArrayEstados))

        self.Verificar_Estados()

        self.declare(States(sn_PIB=sn_PIB))

    @Rule(AND(States(nm_Biome='Mata Atlântica'),
              States(nm_Climate="Subropical"),
              States(sn_Coast="S"),
              States(sn_Border="S"),
              States(sn_PIB="N"),
              ))
    def Print_Parana(self):
        print('O estado em que você está pensando é o Paraná')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Amazonia'),
              States(nm_Climate="Equatorial"),
              States(sn_Coast="S"),
              States(sn_Border="S"),
              States(sn_PIB="S"),
              ))
    def Print_Para(self):
        print('O estado em que você está pensando é o Pará')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Amazonia'),
          States(nm_Climate="Equatorial"),
          States(sn_Coast="S"),
          States(sn_Border="S"),
          States(sn_PIB="N"),
              ))
    def Print_Amapa(self):
        print('O estado em que você está pensando é o Amapá')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Mata Atlântica'),
              States(nm_Climate="Subropical"),
              States(sn_Coast="S"),
              States(sn_Border="S"),
              States(sn_PIB="S"),
              ))
    def Print_Santa_Catarina(self):
        print('O estado em que você está pensando é o Santa Catarina')
        self.declare(Fact(action='restart'))

    ################################################# Population ################################################

    @Rule(AND(
        (States(nm_Biome=W())),
        (States(nm_Climate=W())),
        (States(sn_Coast=W())),
        (States(sn_Border=W())),
        (States(sn_PIB=W())),
        NOT(States(sn_Population=W())),
    ))
    def Get_Population_Question(self):
        vl_Population = States.Get_Medium_Value(self.objArrayEstados, 6)

        sn_Population = input(
            'O seu estado tem a população maior que {0}?\n'.format(vl_Population))

        self.objArrayEstados = list(filter(lambda x: x[6] > vl_Population,  self.objArrayEstados) if sn_Population == 'S' else filter(
            lambda x: x[6] < vl_Population,  self.objArrayEstados))

        self.Verificar_Estados()

        self.declare(States(sn_Population=sn_Population))

    @Rule(AND(States(nm_Biome='Cerrado'),
          States(nm_Climate="Tropical"),
          States(sn_Coast="N"),
          States(sn_Border="N"),
          States(sn_PIB="S"),
          States(sn_Population="S"),
              ))
    def Print_Minas_Gerais(self):
        print('O estado em que você está pensando é o Minas Gerais')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Cerrado'),
          States(nm_Climate="Tropical"),
          States(sn_Coast="N"),
          States(sn_Border="N"),
          States(sn_PIB="S"),
          States(sn_Population="N"),
              ))
    def Print_Distrito_Federal(self):
        print('O estado em que você está pensando é o Distrito Federal')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Mata Atlântica'),
              States(nm_Climate="Tropical"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="N"),
              States(sn_Population="N")
              ))
    def Print_Espirito_Santo(self):
        print('O estado em que você está pensando é o Espiríto Santo')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Cerrado'),
              States(nm_Climate="Tropical"),
              States(sn_Coast="N"),
              States(sn_Border="N"),
              States(sn_PIB="N"),
              States(sn_Population="S")
              ))
    def Print_Goias(self):
        print('O estado em que você está pensando é o Goiás')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Amazonia'),
          States(nm_Climate="Equatorial"),
          States(sn_Coast="N"),
          States(sn_Border="S"),
          States(sn_PIB="N"),
          States(sn_Population="S")
              ))
    def Print_Acre(self):
        print('O estado em que você está pensando é o Acre')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Amazonia'),
          States(nm_Climate="Equatorial"),
          States(sn_Coast="N"),
          States(sn_Border="S"),
          States(sn_PIB="S"),
          States(sn_Population="S")
              ))
    def Print_Amazonas(self):
        print('O estado em que você está pensando é o Amazonas')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Mata Atlântica'),
          States(nm_Climate="Tropical"),
          States(sn_Coast="S"),
          States(sn_Border="N"),
          States(sn_PIB="N"),
          States(sn_Population="S")
              ))
    def Print_Bahia(self):
        print('O estado em que você está pensando é o Bahia')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Cerrado'),
          States(nm_Climate="Tropical"),
          States(sn_Coast="N"),
          States(sn_Border="N"),
          States(sn_PIB="N"),
          States(sn_Population="N")
              ))
    def Print_Tocantins(self):
        print('O estado em que você está pensando é o Tocantins')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Mata Atlântica'),
              States(nm_Climate="Tropical"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="S"),
              States(sn_Population="S")
              ))
    def Print_Sao_Paulo(self):
        print('O estado em que você está pensando é o São Paulo')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Mata Atlântica'),
              States(nm_Climate="Tropical"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="S"),
              States(sn_Population="N")
              ))
    def Print_Rio_de_Janeiro(self):
        print('O estado em que você está pensando é o Rio de Janeiro')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Caatinga'),
              States(nm_Climate="Semiárido"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="S"),
              States(sn_Population="N"),
              ))
    def Print_Rio_Grande_do_Norte(self):
        print('O estado em que você está pensando é o Rio Grande do Norte')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Amazonia'),
              States(nm_Climate="Equatorial"),
              States(sn_Coast="N"),
              States(sn_Border="S"),
              States(sn_PIB="S"),
              States(sn_Population="N")
              ))
    def Print_Rondonia(self):
        print('O estado em que você está pensando é o Rondônia')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Amazonia'),
              States(nm_Climate="Equatorial"),
              States(sn_Coast="N"),
              States(sn_Border="S"),
              States(sn_PIB="N"),
              States(sn_Population="N")
              ))
    def Print_Roraima(self):
        print('O estado em que você está pensando é o Roraima')
        self.declare(Fact(action='restart'))

    ################################################# COVID ################################################

    @Rule(AND(
        (States(nm_Biome=W())),
        (States(nm_Climate=W())),
        (States(sn_Coast=W())),
        (States(sn_Border=W())),
        (States(sn_PIB=W())),
        (States(sn_Population=W())),
        NOT(States(sn_Covid=W())),
    ))
    def Get_COVID_Question(self):
        vl_COVID = States.Get_Medium_Value(self.objArrayEstados, 7)

        sn_Covid = input(
            'O seu estado teve o número de óbitos por COVID  maior que {0}?\n'.format(vl_COVID))

        self.objArrayEstados = list(filter(lambda x: x[7] > vl_COVID,  self.objArrayEstados) if sn_Covid == 'S' else filter(
            lambda x: x[7] < vl_COVID,  self.objArrayEstados))

        self.Verificar_Estados()

        self.declare(States(sn_Covid=sn_Covid))

    @Rule(AND(States(nm_Biome='Caatinga'),
              States(nm_Climate="Semiárido"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="N"),
              States(sn_Population="S"),
              States(sn_Covid="N")
              ))
    def Print_Alagoas(self):
        print('O estado em que você está pensando é o Alagoas')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Caatinga'),
              States(nm_Climate="Semiárido"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="N"),
              States(sn_Population="N"),
              States(sn_Covid="N")
              ))
    def Print_Sergipe(self):
        print('O estado em que você está pensando é o Sergipe')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Caatinga'),
              States(nm_Climate="Semiárido"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="S"),
              States(sn_Population="S"),
              States(sn_Covid="S")
              ))
    def Print_Ceara(self):
        print('O estado em que você está pensando é o Ceará')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Caatinga'),
              States(nm_Climate="Semiárido"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="N"),
              States(sn_Population="S"),
              States(sn_Covid="S")
              ))
    def Print_Paraiba(self):
        print('O estado em que você está pensando é o Paraíba')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Caatinga'),
              States(nm_Climate="Semiárido"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="S"),
              States(sn_Population="S"),
              States(sn_Covid="N")
              ))
    def Print_Pernambuco(self):
        print('O estado em que você está pensando é o Pernambuco')
        self.declare(Fact(action='restart'))

    @Rule(AND(States(nm_Biome='Caatinga'),
              States(nm_Climate="Semiárido"),
              States(sn_Coast="S"),
              States(sn_Border="N"),
              States(sn_PIB="N"),
              States(sn_Population="N"),
              States(sn_Covid="S")
              ))
    def Print_Piaui(self):
        print('O estado em que você está pensando é o Piauí')
        self.declare(Fact(action='restart'))


engine = Classificator()
engine.reset()

engine.declare(Fact(action='start'))

engine.run()


Amazonia 1 [['Acre', 'Amazonia', 'Equatorial', 'N', 'S', 15331, 906876, 1816], ['Alagoas', 'Caatinga', 'Semiárido', 'S', 'N', 54413, 3365351, 6141], ['Amapá', 'Amazonia', 'Equatorial', 'S', 'S', 16795, 877613, 1962], ['Amazonas', 'Amazonia', 'Equatorial', 'N', 'S', 100109, 4269995, 13705], ['Bahia', 'Mata Atlântica', 'Tropical', 'S', 'N', 286240, 14985284, 26650], ['Ceará', 'Caatinga', 'Semiárido', 'S', 'N', 155904, 9240580, 24127], ['Distrito Federal', 'Cerrado', 'Tropical', 'N', 'N', 254817, 3094325, 10222], ['Espiríto Santo', 'Mata Atlântica', 'Tropical', 'S', 'N', 137020, 4108508, 12369], ['Goiás', 'Cerrado', 'Tropical', 'N', 'N', 195682, 7206589, 22974], ['Maranhão', 'Amazonia', 'Equatorial', 'S', 'N', 98179, 7153262, 10106], ['Mato Grosso', 'Pantanal', 'Equatorial', 'N', 'S', 137443, 3567234, 13655], ['Mato Grosso do Sul', 'Pantanal', 'Tropical', 'N', 'S', 106969, 2839188, 9468], ['Minas Gerais', 'Cerrado', 'Tropical', 'N', 'N', 614876, 21411923, 53698], ['Pará', 'Amazonia', 'Equ